NOTE: Make sure to run the script pickle_full_wildfire_data.py to pickle the full 64 x 64 data. The pickled numpy arrays will be used by the Dataset objects to load in the actual data.

In [1]:
import random
import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
import pickle
import numpy as np
import torchvision
from typing import Tuple

# The PyTorch Datasets

In [2]:
def unpickle(f):
    with open(f, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def new_random_crop(labels, new_crop_size):
    crop_map = create_crop_map(len(labels), new_crop_size)
    good_indices = find_good_samples(labels, crop_map, new_crop_size)
    return crop_map, good_indices


def create_crop_map(data_size, crop_size):
    # The crop map assigns an x_shift and y_shift to each sample in the main 64 x 64 dataset.
    crop_map = []
    for i in range(data_size):
        # random.randint returns beginning <= n <= end, hence the minus 1.
        x_shift = random.randint(0, 64 - crop_size - 1)
        y_shift = random.randint(0, 64 - crop_size - 1)
        crop_map.append((x_shift, y_shift))
    return np.array(crop_map)


def get_cropped_sample(index, crop_map, crop_size, data, labels):
    x_shift, y_shift = crop_map[index]
    cropped_features = data[index, :, x_shift : x_shift + crop_size, y_shift : y_shift + crop_size]
    cropped_label = labels[index, x_shift : x_shift + crop_size, y_shift : y_shift + crop_size]
        
    return cropped_features, cropped_label


def find_good_samples(labels, crop_map, crop_size):
    # Finds the indices of samples that have no missing data in their labels.
    # This is determined AFTER generating a crop map and applying the crop to the original 64 x 64 label.
    good_indices = []
    for i in range(len(labels)):
        x_shift, y_shift = crop_map[i]
        if np.all(np.invert(labels[i, x_shift : x_shift + crop_size, y_shift : y_shift + crop_size] == -1)):
            good_indices.append(i)
    return np.array(good_indices)


class WildfireDataset(torch.utils.data.Dataset):
    def __init__(self, data_filename, labels_filename):
        self.data, self.labels = unpickle(data_filename), unpickle(labels_filename)
        self.crop_size = 32
        random.seed(1)
        self.crop_map, self.good_indices = new_random_crop(self.labels, self.crop_size)
        
        print(f"data size: {self.data.nbytes}")
        print(f"label size: {self.labels.nbytes}")
        print(f"crop_map size: {self.crop_map.nbytes}")
        print(f"good_indices size: {self.good_indices.nbytes}")
        print(f"total size: {self.data.nbytes + self.labels.nbytes + self.crop_map.nbytes + self.good_indices.nbytes}")
        print("finished initializing")
        
    def __len__(self):
        return len(self.good_indices)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
            
        index = self.good_indices[index]
        
        cropped_features, cropped_label = get_cropped_sample(index, self.crop_map, self.crop_size, self.data, self.labels)
        
        # Only keep elevation and previous fire mask
        cropped_features = cropped_features[[0, 11], :, :]

        sample = (torch.from_numpy(cropped_features), torch.from_numpy(np.expand_dims(cropped_label, axis=0)))

        return sample


class AugmentedWildfireDataset(torch.utils.data.Dataset):
    def __init__(self, data_filename, labels_filename, transform=None):
        self.data, self.labels = unpickle(data_filename), unpickle(labels_filename)
        self.crop_size = 32
        random.seed(1)
        self.crop_map, self.good_indices = new_random_crop(self.labels, self.crop_size)
        
        self.oversample_indices = self._find_samples_for_oversampling()
        
        print(f"data size: {self.data.nbytes}")
        print(f"label size: {self.labels.nbytes}")
        print(f"crop_map size: {self.crop_map.nbytes}")
        print(f"good_indices size: {self.good_indices.nbytes}")
        print(f"total size: {self.data.nbytes + self.labels.nbytes + self.crop_map.nbytes + self.good_indices.nbytes}")
        print("finished initializing")

    def __len__(self):
        return len(self.good_indices) * 2

    def __getitem__(self, index):
        
        index = self.good_indices[index % len(self.good_indices)] if index < len(self.good_indices) else self._get_random_oversample_index()
            
        cropped_features, cropped_label = get_cropped_sample(index, self.crop_map, self.crop_size, self.data, self.labels)

        # Only keep elevation and previous fire mask
        cropped_features = cropped_features[[0, 11], :, :]

        # Perform random rotations
        rotations = [0, 90, 180, 270]
        random_rotation = random.choice(rotations)
        cropped_features = torchvision.transforms.functional.rotate(torch.from_numpy(cropped_features), random_rotation)
        cropped_label = torchvision.transforms.functional.rotate(torch.from_numpy(np.expand_dims(cropped_label, axis=0)), random_rotation)

        sample = (cropped_features, cropped_label)
        
        return sample
    
    def _find_samples_for_oversampling(self):
        oversample_indices = []
        threshold = 0.05 # Desired percentage of fire pixels in the target fire masks
        
        #print(len(self.good_indices))
        for i in range(len(self.good_indices)):
            index = self.good_indices[i]
            x_shift, y_shift = self.crop_map[index]
            cropped_label = self.labels[index, x_shift : x_shift + self.crop_size, y_shift : y_shift + self.crop_size]
            unique_target, counts_target = np.unique(cropped_label, return_counts=True)
            target_counts_map = {int(unique_target[j]): int(counts_target[j]) for j in range(len(unique_target))}
            
            for key, value in target_counts_map.items():
                if key == 1 and (value/1024) >= threshold:
                    oversample_indices.append(index)
        #print(len(oversample_indices))
        #print(oversample_indices)
        return oversample_indices
        
    
    def _get_random_oversample_index(self):
        return random.choice(self.oversample_indices)

In [3]:
TEST_DATA_FILENAME = '../data/next-day-wildfire-spread/test.data'
TEST_LABELS_FILENAME = '../data/next-day-wildfire-spread/test.labels'

test_dataset = WildfireDataset(TEST_DATA_FILENAME, TEST_LABELS_FILENAME)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

print(len(test_loader))
print(len(test_dataset))

data size: 67485696
label size: 5623808
crop_map size: 21968
good_indices size: 10984
total size: 73142456
finished initializing
14
1373


In [4]:
batch = next(iter(test_loader))
images, labels = batch

print(images.shape)
print(labels.shape)

original_test_data = unpickle(TEST_DATA_FILENAME)
original_test_labels = unpickle(TEST_LABELS_FILENAME)
original_test_labels = np.expand_dims(original_test_labels, axis=1)

print(original_test_data.shape)
print(original_test_labels.shape)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jack/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/jack/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/home/jack/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 175, in default_collate
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "/home/jack/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 175, in <listcomp>
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "/home/jack/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 140, in default_collate
    out = elem.new(storage).resize_(len(bat

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

TITLES = [
    'Elevation',
    'Wind\ndirection',
    'Wind\nvelocity',
    'Min\ntemp',
    'Max\ntemp',
    'Humidity',
    'Precip',
    'Drought',
    'Vegetation',
    'Population\ndensity',
    'Energy\nrelease\ncomponent',
    'Previous\nfire\nmask',
    'Fire\nmask',
    'Predicted\nfire\nmask'
]

CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.1, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)

def imshow(sample, titles):
    fig = plt.figure(figsize=(15,6.5))
    features = sample[0].shape[0]
    for i in range(features + 1):
        plt.subplot(1, features + 1, i + 1)
        plt.title(titles[i], fontsize=13)
        if i < features - 1:
            plt.imshow(sample[0][i, :, :], cmap='viridis')
        if i == features - 1:
            plt.imshow(sample[0][-1, :, :], cmap=CMAP, norm=NORM)
        if i == features:
            plt.imshow(sample[1][0, :, :], cmap=CMAP, norm=NORM)
        plt.axis('off')
    plt.tight_layout()

imshow((images[0], labels[0]), ["Elevation", "Previous\nfire\nmask", "Fire\nmask"])

In [ ]:
test_image = torchvision.transforms.functional.rotate(images[0], 90)
test_label = torchvision.transforms.functional.rotate(labels[0], 90)

imshow((test_image, test_label), ["Elevation", "Previous\nfire\nmask", "Fire\nmask"])

In [ ]:
test_image = torchvision.transforms.functional.rotate(images[0], 180)
test_label = torchvision.transforms.functional.rotate(labels[0], 180)

imshow((test_image, test_label), ["Elevation", "Previous\nfire\nmask", "Fire\nmask"])

In [ ]:
test_image = torchvision.transforms.functional.rotate(images[0], 270)
test_label = torchvision.transforms.functional.rotate(labels[0], 270)

imshow((test_image, test_label), ["Elevation", "Previous\nfire\nmask", "Fire\nmask"])

In [ ]:
imshow((original_test_data[0], original_test_labels[0]), TITLES)

# The Neural Networks

In [ ]:
# Neural networks that can be used on the dataset
# Make sure the training data is scrubbed of any target fire masks that have missing data

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.linear(x))
        x = x.reshape(-1, 1, 32, 32)
        return x

class BinaryClassifierCNN(torch.nn.Module):
    def __init__(self, image_size):
        flattened_conv2_output_dimensions = (image_size//4)**2
        super(BinaryClassifierCNN, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(12, 16, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.out = torch.nn.Sequential(
            torch.nn.Linear(32 * flattened_conv2_output_dimensions, 1024), # 1024 pixels, output represents probability of fire.
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.out(x)
        x = x.reshape(-1, 1, 32, 32)
        return x
    

# Tutorial for the autoencoder: https://www.youtube.com/watch?v=345wRyqKkQ0
class Reshape(torch.nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args
    
    def forward(self, x):
        return x.view(self.shape)
    
class Trim(torch.nn.Module):
    def __init__(self, *args):
        super(Trim, self).__init__()
    
    def forward(self, x):
        return x[:, :, :32, :32]

class ConvolutionalAutoencoder(torch.nn.Module):
    def __init__(self):
        super(ConvolutionalAutoencoder, self).__init__()
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(12, 16, 3, 1, 0), # 32 x 32 -> 30 x 30
            torch.nn.LeakyReLU(0.01),
            torch.nn.Dropout(0.1),
            torch.nn.Conv2d(16, 32, 3, 2, 0), # 30 x 30 -> 14 x 14
            torch.nn.LeakyReLU(0.01),
            torch.nn.Dropout(0.1),
            torch.nn.Conv2d(32, 32, 3, 2, 0), # 14 x 14 -> 6 x 6
            torch.nn.Flatten(),
            torch.nn.Linear(1152, 2) # 1152 = 32 * 6  * 6
        )
        
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(2, 1152),
            Reshape(-1, 32, 6, 6),
            torch.nn.ConvTranspose2d(32, 32, 3, 1, 0), # 6 x 6 -> 8 x 8
            torch.nn.LeakyReLU(0.01),
            torch.nn.ConvTranspose2d(32, 16, 3, 2, 1), # 8 x 8 -> 15 x 15
            torch.nn.LeakyReLU(0.01),
            torch.nn.ConvTranspose2d(16, 16, 3, 2, 0), # 15 x 15 -> 31 x 31
            torch.nn.LeakyReLU(0.01),
            torch.nn.ConvTranspose2d(16, 1, 3, 1, 0), # 31 x 31 -> 33 x 33
            Trim(),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Code from: https://github.com/milesial/Pytorch-UNet


""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


 

""" Full assembly of the parts to form the complete network """


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)

# Loading in the model

In [ ]:
MODEL_PATH = "../savedModels/model-UNet-bestLoss-Rank-0.weights"
model = UNet(2, 1, True)

#https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/2
# original saved file with DataParallel
state_dict = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)

model.eval()

In [ ]:
train_loss_history = unpickle("../savedModels/model-UNet-train-loss-Rank-0.history")
validation_loss_history = unpickle("../savedModels/model-UNet-validation-loss-Rank-0.history")

In [ ]:
plt.plot(train_loss_history, linestyle="solid")
plt.plot(validation_loss_history, linestyle="solid")

In [ ]:
train_loss_history = unpickle("../savedModels/model-UNet-train-loss-Rank-1.history")
validation_loss_history = unpickle("../savedModels/model-UNet-validation-loss-Rank-1.history")

In [ ]:
plt.plot(train_loss_history, linestyle="solid")
plt.plot(validation_loss_history, linestyle="solid")

# Testing the loaded in model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

def test_model(test_loader, model):   
    total = 0
    correct = 0
    incorrect = 0
    
    y_pred = []
    y_true = []
    
    model.eval()
    
    with torch.no_grad():
        
        for i, (batch_inputs, batch_labels) in enumerate(test_loader):
            out = model(batch_inputs)
            
            targets = torch.flatten(batch_labels)
            out = torch.flatten(out)
            
            fire_predictions = torch.round(torch.sigmoid(out))
            
            y_pred.extend(fire_predictions)
            y_true.extend(targets)
            
            correct += torch.sum(fire_predictions == targets)
            incorrect += torch.sum(fire_predictions != targets)
            
            total += len(targets)
    
    accuracy = 100 * correct/total
    print(f"Total = {total}")
    print(f"This should match the total: {correct + incorrect}")
    print(f"Total correct = {correct}")
    print(f"Total incorrect = {incorrect}")
    print(f"Overall accuracy: {accuracy}")
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    
    print("\nPrecision, Recall, Fscore, Support:")
    print(np.vstack(precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])))

In [ ]:
test_model(test_loader, model)

# Plotting the loaded in model

In [ ]:
TITLES = [
    'Elevation',
    'Wind\ndirection',
    'Wind\nvelocity',
    'Min\ntemp',
    'Max\ntemp',
    'Humidity',
    'Precip',
    'Drought',
    'Vegetation',
    'Population\ndensity',
    'Energy\nrelease\ncomponent',
    'Previous\nfire\nmask',
    'Fire\nmask',
    'Predicted\nfire\nmask'
]

def plot_5_predictions(loader, model, offset, titles):
    with torch.no_grad():
        batch = next(iter(loader))
        out = model(batch[0])
        fire_predictions = torch.squeeze(torch.round(torch.sigmoid(out)), dim=1).numpy()
        
        print(fire_predictions.shape)
    
        n_rows = 5
        n_features = batch[0].shape[1]
    
        CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
        BOUNDS = [-1, -0.1, 0.1, 1]
        NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
    
        fig = plt.figure(figsize=(15,6.5))
    
    for i in range(n_rows):
        for j in range(n_features + 2):
            plt.subplot(n_rows, n_features + 2, i * (n_features + 2) + j + 1)
            if i == 0:
                plt.title(titles[j], fontsize=13)
            if j < n_features - 1:
                plt.imshow(batch[0][i+offset, j, :, :], cmap='viridis')
            if j == n_features - 1:
                plt.imshow(batch[0][i+offset, -1, :, :], cmap=CMAP, norm=NORM)
            if j == n_features:
                plt.imshow(batch[1][i+offset, 0, :, :], cmap=CMAP, norm=NORM)
            if j > n_features:
                plt.imshow(fire_predictions[i+offset], cmap=CMAP, norm=NORM)
            plt.axis('off')
    plt.tight_layout()
    

In [ ]:
test_titles = ["Elevation", "Previous\nfire\nmask", "Fire\nmask", "Predicted\nfire\mask"]

plot_5_predictions(test_loader, model, offset=15, titles=test_titles)